# About: パラメータ設定

---

Moodle環境を構築するのに必要となるパラメータを設定します。

## 全体構成

VCP SDKを用いてクラウド上に仮想マシンを作成し、そこにMoodle環境を構築します。

![moodleの構成](images/moodle-000-01.png)

この一連のNotebookではDockerコンテナを用いて[Moodle](https://moodle.org/)環境の構築を行います。

構築するMoodle環境は次の３つのコンテナで構成されています。

* Moodle
  - MoodleのPHPスクリプトを実行するコンテナ
* [MariaDB](https://hub.docker.com/_/mariadb/)
  - Moodleの設定を保存するデータベース
* [Apache HTTP Server](https://hub.docker.com/_/httpd/)
  - Moodleサーバに対するリバースプロキシ

ここでは小規模構成を想定して３つのコンテナを１つのVCノード（仮想マシン）に配置する構成とします。
またMoodleとMariaDBコンテナに対しては、データを保存するためのVCディスク（仮想ディスク）をそれぞれのコンテナに割り当てます。

## VCP SDK

VCP SDKを利用する際に必要となるパラメータを設定します。

![VCP SDK](images/moodle-010-02.png)

### VCCアクセストークンの入力

VCノードを起動するにはVC Controller(VCC)にアクセスして、操作を行う必要があります。VCCにアクセスするために必要となるアクセストークンをここで入力します。

次のセルを実行すると入力枠が表示されるのでアクセストークンの値を入力してください。

> アクセストークン入力後に Enter キーを押すことで入力が完了します。

In [ ]:
from getpass import getpass
vcc_access_token = getpass()

入力されたアクセストークンが正しいことを、実際にVCCにアクセスして確認します。

In [ ]:
from vcpsdk.vcpsdk import VcpSDK
vcp = VcpSDK(vcc_access_token)

上のセルの実行結果がエラーとなり以下のようなメッセージが表示されている場合は、入力されたアクセストークンに誤りがあります。

```
config vc failed: http_status(403)
2020/XX/XX XX:XX:XX UTC: VCPAuthException: xxxxxxx:token lookup is failed: permission denied
```

エラーになった場合はこの節のセルを全て `unfreeze` してから、もう一度アクセストークンの入力を行ってください。

> `unfreeze`するにはNotebookのツールバーにあるボタン<i class="fa-unfreeze fa"></i>、<i class="fa-unfreeze-below-in-section fa"></i>、<i class="fa-unfreeze-below-all fa"></i> を利用してください。

### UnitGroup名の指定

このアプリケーションテンプレートで構築するMoodle環境に対して名前を付けます。指定した名前はVCPのUnitGroup名となります。

VCPの構成要素は以下のようになっています。

* VCノード
  - クラウドにおける計算資源(VM/BM)
  - 例えば Amazon EC2インスタンス, Microsoft Azure VM など
* Unit
  - 同質のVCノードにより構成されている要素
  - 同じUnitに属するVCノードはCPU,メモリ等の計算資源が全て同じ設定になっている
* UnitGroup
  - 複数のUnitにより構成されている要素
  - 使用目的、ライフサイクルなどに合わせて、複数のUnitをまとめて扱うための要素  

VCP SDKで作成した他の環境と区別するために UnitGroupに名前を付けます。UnitGroup名は既存のものと異なる値を指定する必要があります。
既存のUnitGroupを確認するために現在のUnitGroupの一覧を表示します。

In [ ]:
vcp.df_ugroups()

ここで構築する環境のUnitGroup名を次のセルで指定してください。

> UnitGroup名はアルファベットで開始され、その後に英数字またはアンダースコア`_`からなる文字列を指定してください。

In [ ]:
# (例)
# ugroup_name = 'Moodle'

ugroup_name = 

### パラメータの保存

この章で指定したパラメータの値をファイルに保存します。

後の手順でVCノードに対する操作を、構成管理ツールの[Ansible](https://www.ansible.com/)で行います。そこで、パラメータの保存形式は `Ansible` のフォーマットに従うことにします。Ansible では `group_vars/`というディレクトリに YAML フォーマットのファイルを配置すると、そのファイルに記録されている値を変数として利用することができます。このNotebookでは `group_vars/` にあるファイルを `group_vars ファイル`と呼ぶことにします。

値の保存を行う前に、入力されたパラメータに対して簡易なチェックを行います。エラーになった場合はその後に表示される指示に従ってください。

In [ ]:
%run scripts/utils.py

check_parameters(
    'ugroup_name',
    params={'vcp': vcp},
    nb_vars=locals(),
)

次のセルを実行すると、この章で指定したパラメータが group_vars ファイルに保存されます。

In [ ]:
%run scripts/group.py
from pathlib import Path

update_group_vars(
    ugroup_name,
    ugroup_name=ugroup_name,
)

`group_vars`ファイルの内容を確認してみます。

In [ ]:
!cat group_vars/{ugroup_name}.yml

## VCノード

Moodle環境を構築するVCノード（仮想マシン）に関するパラメータを指定します。

![VCノード設定](images/moodle-010-03.png)

### クラウドプロバイダの指定

VCノードを起動するプロバイダを選択します。

In [ ]:
# (例)
# vc_provider = 'aws'
# vc_provider = 'azure'

vc_provider = 

### VCノードに割り当てるリソース量の指定


VCノードに割り当てるリソース量を`spec`オブジェクトに指定します。

VCノードの`spec`オブジェクトに対して全てのパラメータを毎回設定するのは煩雑です。そのためVCP SDKでは典型的な構成のパラメータセットを事前に定義してあります。事前に定義した`spec`パラメータセットのことを`flavor`と呼び、`spec`を作成する際に指定することになっています。

`spec`に設定できるパラメータはクラウドプロバイダ毎に異なるので 各`flavor`に対応するパラメータの値もプロバイダ毎に定義されています。
次のセルを実行すると `vc_provider` に設定したプロバイダに対応する `flavor` の一覧が表示されます。

In [ ]:
vcp.df_flavors(vc_provider)

表示された `flavor` の値から一つを選択して、次のセルに指定してください。

In [ ]:
# (例)
# vc_flavor = 'small'

vc_flavor = 

`flavor`に設定されている以外のインスタンスタイプ(VMタイプ)を指定したい場合は、次のセルで`vc_vm_type`の値を指定してください。

> AWS, Azure の場合の設定例をコメントに記しています。必要に応じてコメントを外して設定を行ってください。

In [ ]:
# (例)
# vc_vm_type = 'm5.large'          # AWS
# vc_vm_type = 'Standard_D4_v3'    # Azure

### IPアドレスの指定

Moodleを実行するVCノードはHTTPサーバとなるので、事前に定めたIPアドレスを割り当てたほうが構築しやすくなります。
ここではVCノードのIPアドレスを指定します。

まず指定可能なIPアドレスの範囲を確認します。次のセルを実行するとVCノードに割り当て可能なIPアドレスの範囲が表示されます。

In [ ]:
print(vcp.get_vpn_catalog(vc_provider).get('private_network_ipmask'))

次のセルでVCノードに割り当てる （プライベート）IPアドレスを指定してください。

In [ ]:
# (例)
# vc_moodle_ipaddress = '172.30.2.100'

vc_moodle_ipaddress = 

### SSH公開鍵認証の鍵ファイルの指定

起動したVCノードにsshでログインして操作を行うために、事前にSSH公開鍵認証の公開鍵を登録する必要があります。

VCノードに登録するSSHの公開鍵ファイルのパスを次のセルで指定してください。

In [ ]:
# (例)
# ssh_public_key_path = '~/.ssh/id_rsa.pub'

ssh_public_key_path = 

公開鍵に対応する秘密鍵のパスを次のセルで指定してください。

In [ ]:
# (例)
# ssh_private_key_path = '~/.ssh/id_rsa'

ssh_private_key_path = 

### パラメータの保存

この章で指定したパラメータの値をファイルに保存します。

値の保存を行う前に、入力されたパラメータに対して簡易なチェックを行います。エラーになった場合はその後に表示される指示に従ってください。

In [ ]:
%run scripts/utils.py

check_parameters(
    'vc_provider',
    'vc_flavor',
    'vc_moodle_ipaddress',
    'ssh_public_key_path',
    'ssh_private_key_path',
    params={'vcp': vcp},
    nb_vars=locals(),
)

次のセルを実行すると、この章で指定したパラメータが group_vars ファイルに保存されます。

In [ ]:
%run scripts/group.py

update_group_vars(
    ugroup_name,
    vc_provider=vc_provider,
    vc_flavor=vc_flavor,
    vc_moodle_ipaddress=vc_moodle_ipaddress,
    ssh_public_key_path=str(ssh_public_key_path),
    ssh_private_key_path=str(ssh_private_key_path),
)
if 'vc_vm_type' in vars():
    update_group_vars(ugroup_name, vc_vm_type=vc_vm_type)

ここまでに保存したパラメータを確認するために group_vars ファイルの内容を表示します。

In [ ]:
!cat group_vars/{ugroup_name}.yml

## Moodle
Moodleのバージョン等の、直接Moodleに関わるパラメータを指定します。

![Moodle設定](images/moodle-010-04.png)

指定するパラメータのうちMoodleの管理者パスワードについては秘密情報となるので、ローカル環境（Notebook環境）には保存せず、VCCが提供している HashiCorp Vault に保存します。HashiCorp Vault は秘密情報を保存するための Key Valueストアです。保持する情報は暗号化されます。

秘密情報ではないその他のパラメータについては、前章と同様にローカル環境の group_vars ファイルに記録します。

### Moodleのバージョン

インストールするMoodleの[バージョン](https://docs.moodle.org/dev/Releases)を指定します。

> ここで指定される値は[Gitのタグ名](https://github.com/moodle/moodle/tags)であることを想定しています。そのため、必ず先頭に `v` を付けてください。 

In [ ]:
# (例)
# moodle_version = 'v3.9.9'
# moodle_version = 'v3.11.2'
# moodle_version = 'v3.10.6'

moodle_version = 

### Moodleの管理者ユーザ名

Moodleの管理者ユーザ名を指定します。

> Moodleのユーザ名に指定できる文字は、小文字英数字と`_`, `-`, `@`, `.`です。

In [ ]:
# (例)
# moodle_admin_name = 'admin'

moodle_admin_name = 'admin'

### Moodleの管理者パスワード

Moodleの管理者ユーザのパスワードを次のセルで指定してください。

> 自明なパスワード（`admin`　または空文字）を設定することはできません。 

In [ ]:
from getpass import getpass
moodle_admin_password = getpass()

### Moodleのコンテナイメージ

Moodleコンテナのイメージ名を指定してください。

指定できる値を以下に示します。

* harbor.vcloud.nii.ac.jp/vcp/moodle:php74
* harbor.vcloud.nii.ac.jp/vcp/moodle:php73

上記のコンテナイメージはインストールされているPHPのバージョンが異なります。それぞれ PHP 7.4, PHP 7.3 がインストールされています。

In [ ]:
moodle_image_name = 'harbor.vcloud.nii.ac.jp/vcp/moodle:php74'

### Moodleのディスクサイズ

このNotebookで構築する環境では、Moodleコンテナ用に専用のディスクを用意します。ディスクに割り当てるサイズ(GB)を次のセルで指定してください。

In [ ]:
# (例)
# moodle_disk_size = 100

moodle_disk_size = 

### Moodleのボリュームサイズ

このNotebookで構築する環境ではディスクを直接利用するのではなく、LVMを用いて用途ごとに論理ボリューム(LV)を作成してそのボリュームを利用するようにします。また、作成する論理ボリュームは全てシンプロビジョニングされたものとします。そのため論理ボリュームの容量は、実際のディスク容量に依存しない仮想化した値を設定できます。

Moodle用のディスクでは、データ(moodledata)とPHPファイルを格納する論理ボリュームを作成します。それぞれのボリュームサイズを指定してください。

データ(moodledata)用論理ボリュームのサイズ(GB)を指定してください。

In [ ]:
# (例)
# moodle_volume_data_size = 100

moodle_volume_data_size = 

MoodleのPHPファイル用論理ボリュームのサイズ(GB)を指定してください。

In [ ]:
# (例)
# moodle_volume_php_size = 5

moodle_volume_php_size = 

### Moodleボリュームの暗号化

必要に応じてMoodleボリュームに対して [LUKS](https://gitlab.com/cryptsetup/cryptsetup) を用いた暗号化を行うことができます。

暗号化を行う場合は次のセルで`moodle_volume_encrypt` の値を `True` に設定してください。ボリュームの暗号化を行わない場合は `False` を設定してください。

In [ ]:
# (例)
# moodle_volume_encrypt = True   # 暗号化する場合
# moodle_volume_encrypt = False  # 暗号化しない場合

moodle_volume_encrypt = 

ボリュームの暗号化を行う場合はキーの生成を行います。

In [ ]:
if moodle_volume_encrypt:
    from secrets import token_bytes
    from base64 import b64encode
    
    moodle_volume_key = b64encode(token_bytes(256))

### MoodleのURL

MoodleではサイトのURLを`config.php`の`$CFG->wwwroot`に設定する必要があります。構築対象のURLを指定してください。

In [ ]:
# (例)
# moodle_url = 'https://moodle.example.org'

moodle_url = 

### パラメータの保存

この章で指定したパラメータの値をファイルに保存します。

内容によって異なる場所に保存します。パスワード以外の秘密情報ではないパラメータについては前の章と同様にJupyter Notebook環境内の group_vars ファイルに保存します。パスワード等の秘密情報については、より安全度の高いVCCのVaultサーバに保存します。

値の保存を行う前に、入力されたパラメータに対して簡易なチェックを行います。エラーになった場合はその後に表示される指示に従ってください。

In [ ]:
%run scripts/utils.py

check_parameters(
    'moodle_version',
    'moodle_admin_name',
    'moodle_admin_password',
    'moodle_image_name',
    'moodle_disk_size',
    'moodle_volume_data_size',
    'moodle_volume_php_size',
    'moodle_volume_encrypt',
    'moodle_url',
    params={'vcp': vcp},
    nb_vars=locals(),
)

#### MoodleパラメータのVaultサーバへの保存

Moodleの管理者パスワードをVCCのVaultサーバに保存します。

Vaultサーバ内の保存場所のパスを指定します。

> 他の秘密情報の保存場所と重複しないようにするために UnitGroup 名をパスに含めています。

In [ ]:
moodle_vault_path = f'cubbyhole/{ugroup_name}/moodle/admin'
print(moodle_vault_path)

Vaultサーバに秘密情報を保存します。

> 次のセルでは Vault の REST API を呼び出すために Python の requests ライブラリを利用しています。Vaultサーバの REST APIや requests ライブラリの簡単な利用法については vcp tutorialの[「102-秘密情報管理サーバの利用.ipynb」](https://github.com/nii-gakunin-cloud/handson/blob/master/handson201902-vcp/102-%E7%A7%98%E5%AF%86%E6%83%85%E5%A0%B1%E7%AE%A1%E7%90%86%E3%82%B5%E3%83%BC%E3%83%90%E3%81%AE%E5%88%A9%E7%94%A8.ipynb) を参照してください。

In [ ]:
import requests
import time

# 保存するパラメータを１つの dict にまとめる
payload = {
    'password': moodle_admin_password,
}
if moodle_volume_encrypt:
    payload['moodle_volume_key'] = moodle_volume_key
    payload['moodle_volume_key_' + time.strftime("%Y%m%d%H%M%S")] = moodle_volume_key

# Vault サーバのURLを組み立てる
url = f'{vcp.vcc_info()["vault_url"]}/v1/{moodle_vault_path}'
print(url)

# VaultサーバのアクセストークンをHTTPヘッダに設定する
custom_headers = {
    'X-Vault-Token': vcc_access_token,
}

r = requests.post(url, headers=custom_headers, json=payload)
r

Vaultサーバへの値の保存が成功すると　`<Response [204]>` と表示されます。

#### group_vars ファイルへの保存

秘密情報以外のパラメータを`group_vars`ファイルに保存します。

Vaultサーバの保存先パスが設定されていることを確認します。

In [ ]:
%run scripts/utils.py

check_parameters(
    'moodle_vault_path',
    params={'vcp': vcp},
    nb_vars=locals(),
)

次のセルを実行すると、この章で指定した秘密情報以外のパラメータが group_vars ファイルに保存されます。

In [ ]:
%run scripts/group.py

update_group_vars(
    ugroup_name,
    moodle_version=moodle_version,
    moodle_admin_name=moodle_admin_name,
    moodle_image_name=moodle_image_name,
    moodle_disk_size=moodle_disk_size,
    moodle_volume_data_size=moodle_volume_data_size,
    moodle_volume_php_size=moodle_volume_php_size,
    moodle_volume_encrypt=moodle_volume_encrypt,
    moodle_url=moodle_url,
    moodle_vault_path=moodle_vault_path,
)

ここまでに保存したパラメータを確認するために group_vars ファイルの内容を表示します。

In [ ]:
!cat group_vars/{ugroup_name}.yml

## データベース

Moodleの設定値などを保存するデータベースに関するパラメータを指定します。

![データベース設定](images/moodle-010-05.png)

指定するパラメータのうちデータベースに接続するためのパスワードについては秘密情報となるので、ローカル環境（Notebook環境）には保存せず、VCCが提供している HashiCorp Vault に保存します。

秘密情報ではないその他のパラメータについては、前章と同様にローカル環境の group_vars ファイルに記録します。

### データベースのコンテナイメージ

データベースコンテナのイメージ名を指定してください。このNotebookが構築する環境では MariaDBのコンテナイメージを指定することができます。

In [ ]:
db_image_name = 'mariadb:10.6'

### データベース名

Moodleが使用するデータベース名を指定してください。

In [ ]:
db_moodle_db = 'moodle'

### データベースの接続ユーザ

Moodleのデータベースに接続するためのデータベースのユーザ名を指定してください。

In [ ]:
db_moodle_db_user = 'moodle'

### データベースのパスワード

Moodleのデータベースに接続するためのパスワードを指定します。

In [ ]:
from getpass import getpass
db_moodle_db_password = getpass()

### データベースのディスクサイズ

このNotebookで構築する環境では、データベース用に専用のディスクを用意します。ディスクに割り当てるサイズ(GB)を次のセルで指定してください。

In [ ]:
# (例)
# db_disk_size = 12

db_disk_size = 

### データベースのボリュームサイズ

このNotebookで構築する環境ではディスクを直接利用するのではなく、LVMを用いて論理ボリューム(LV)を作成してそのボリュームを利用するようにします。論理ボリュームを作成することにより、LVMのスナップショット機能や、作成後のサイズ拡張などの機能が利用可能になります。また、作成する論理ボリュームは全てシンプロビジョニングされたものとします。そのため論理ボリュームの容量は、実際のディスク容量に依存しない仮想化した値を設定できます。

データベースとして使用するボリュームサイズ(GB)を指定してください。

In [ ]:
db_volume_size = db_disk_size

### データベースボリュームの暗号化

必要に応じてデータベースボリュームに対して [LUKS](https://gitlab.com/cryptsetup/cryptsetup) を用いた暗号化を行うことができます。

暗号化を行う場合は次のセルで`db_volume_encrypt` の値を `True` に設定してください。ボリュームの暗号化を行わない場合は `False` を設定してください。

> 次のセルの始めの記述ではMoodleボリュームの暗号化設定と同じ値を設定しています。

In [ ]:
# (例)
# db_volume_encrypt = True   # 暗号化する場合
# db_volume_encrypt = False  # 暗号化しない場合

db_volume_encrypt = moodle_volume_encrypt

ボリュームの暗号化を行う場合はキーの生成を行います。

In [ ]:
if db_volume_encrypt:
    from secrets import token_bytes
    from base64 import b64encode
    
    db_volume_key = b64encode(token_bytes(256))

### パラメータの保存

この章で指定したパラメータの値をファイルに保存します。

内容によって異なる場所に保存します。パスワード以外の秘密情報ではないパラメータについては前の章と同様にJupyter Notebook環境内の group_vars ファイルに保存します。パスワード等の秘密情報については、より安全度の高いVCCのVaultサーバに保存します。

#### データベースパラメータのVaultサーバへの保存

データベースの接続パスワードをVCCのVaultサーバに保存します。

Vaultサーバ内の保存場所のパスを指定します。

> 他の秘密情報の保存場所と重複しないようにするために UnitGroup 名をパスに含めています。

In [ ]:
db_vault_path = f'cubbyhole/{ugroup_name}/db/moodle'
print(db_vault_path)

Vaultサーバに秘密情報のパラメータを保存します。

In [ ]:
import requests
import time

# 保存するパラメータを１つの dict にまとめる
payload = {
    'password': db_moodle_db_password,
}
if db_volume_encrypt:
    payload['db_volume_key'] = db_volume_key
    payload['db_volume_key_' + time.strftime("%Y%m%d%H%M%S")] = db_volume_key

# Vault サーバのURLを組み立てる
url = f'{vcp.vcc_info()["vault_url"]}/v1/{db_vault_path}'
print(url)

# VaultサーバのアクセストークンをHTTPヘッダに設定する
custom_headers = {
    'X-Vault-Token': vcc_access_token,
}

r = requests.post(url, headers=custom_headers, json=payload)
r

Vaultサーバへの値の保存が成功すると　`<Response [204]>` と表示されます。

#### group_vars ファイルへの保存

秘密情報以外のパラメータを`group_vars`ファイルに保存します。

値の保存を行う前に、入力されたパラメータに対して簡易なチェックを行います。エラーになった場合はその後に表示される指示に従ってください。

In [ ]:
%run scripts/utils.py

check_parameters(
    'db_image_name',
    'db_moodle_db',
    'db_moodle_db_user',
    'db_disk_size',
    'db_volume_size',
    'db_volume_encrypt',
    'db_vault_path',
    params={'vcp': vcp},
    nb_vars=locals(),
)

次のセルを実行すると、この章で指定した秘密情報以外のパラメータが group_vars ファイルに保存されます。

In [ ]:
update_group_vars(
    ugroup_name,
    db_image_name=db_image_name,
    db_moodle_db=db_moodle_db,
    db_moodle_db_user=db_moodle_db_user,
    db_disk_size=db_disk_size,
    db_volume_size=db_volume_size,
    db_volume_encrypt=db_volume_encrypt,
    db_vault_path=db_vault_path,
)

group_vars ファイルの内容を表示して保存されたパラメータを確認します。

In [ ]:
!cat group_vars/{ugroup_name}.yml

## リバースプロキシ

Moodleコンテナの前でリクエストを受け付けるリバースプロキシに関するパラメータを指定します。

![リバースプロキシ設定](images/moodle-010-06.png)

このNotebookで構築するMoodle環境では、利用者からのアクセスをMoodleコンテナが直接受け付けるのではなく、その前段にリバースプロキシを設置します。リバースプロキシの利用目的は主に以下の２つです。

* リクエスト内容に応じた処理の切り替え
    + 運用環境と検証環境など複数の環境を、リクエスト元のIPアドレスなどで切り替える
* SSL/TLS
    + Moodleコンテナが直接 SSL/TLS を行うかわりに、リバースプロキシが SSL/TLS 接続を行う

Apache HTTP Serverをリバースプロキシとして利用します。

### リバースプロキシのコンテナイメージ

リバースプロキシとして実行する、Apache HTTP Serverコンテナのイメージ名を指定してください。

In [ ]:
rproxy_image_name = 'httpd:2.4'

### サーバ証明書

SSL/TLS のために使用するサーバ証明書と秘密鍵のファイルを指定してください。

> 「000-README.ipynb」に記したようにサーバ証明書と秘密鍵は事前にNotebook環境にアップロードしてあると想定しています。ここではアップロードしたファイルのパスを指定してください。

サーバ証明書のパスを指定してください。

In [ ]:
# (例)
# rproxy_tls_cert_path = 'cert/server.crt'

rproxy_tls_cert_path = 

秘密鍵のパスを指定してください。

In [ ]:
# (例)
# rproxy_tls_key_path = 'cert/server.key'

rproxy_tls_key_path = 

### パラメータの保存

この章で指定したパラメータの値をファイルに保存します。

値の保存を行う前に、入力されたパラメータに対して簡易なチェックを行います。エラーになった場合はその後に表示される指示に従ってください。

In [ ]:
%run scripts/utils.py

check_parameters(
    'rproxy_image_name',
    'rproxy_tls_cert_path',
    'rproxy_tls_key_path',
    params={'vcp': vcp},
    nb_vars=locals(),
)

次のセルを実行すると、この章で指定したパラメータが group_vars ファイルに保存されます。

In [ ]:
update_group_vars(
    ugroup_name,
    rproxy_image_name=rproxy_image_name,
    rproxy_tls_cert_path=rproxy_tls_cert_path,
    rproxy_tls_key_path=rproxy_tls_key_path,
)

group_vars ファイルの内容を表示して保存されたパラメータを確認します。

In [ ]:
!cat group_vars/{ugroup_name}.yml

## チェックとデフォルト値の保存

全てのパラメータが設定されたことをチェックします。

In [ ]:
%run scripts/group.py
gvars = load_group_vars(ugroup_name)
require_params = [
    'ugroup_name', 'vc_provider', 'vc_flavor', 'vc_moodle_ipaddress',
    'ssh_private_key_path', 'ssh_public_key_path',
    'moodle_version', 'moodle_admin_name', 'moodle_image_name',
    'moodle_disk_size', 'db_disk_size',
    'moodle_volume_data_size', 'moodle_volume_php_size', 'db_volume_size',
    'moodle_vault_path', 'db_vault_path',
    'db_image_name', 'db_moodle_db', 'db_moodle_db_user',
    'rproxy_tls_cert_path', 'rproxy_tls_key_path', 'rproxy_image_name',
    'moodle_url',
]

for x in require_params:
    if x not in gvars:
        raise RuntimeError("ERROR: not set {}".format(x))

デフォルト値を設定する必要があるパラメータを保存します。

In [ ]:
gvars = load_group_vars(ugroup_name)
if 'project_tag' not in gvars:
    update_group_vars(ugroup_name, project_tag='0')